# *IN PROGRESS

First things first, we want to get our stock data. We will be predicting the 'SPY' index ticker using Yahoo Finance as it is the most readily available free source of stock data. 

    Ideally, we would want to use ticker data in order to create the most optimal model, but that is usually locked behind a broker's database for a fee.

Pandas has a neat model called DataReader that makes scraping Stock price points a breeze:

In [28]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_datareader import data

panel_data = data.DataReader('SPY', 'yahoo', '2017-01-01', '2022-06-14')

This code pulls a DataFrame that can be thought of as a 3D matrix. The first dimension consists of the various fields Yahoo Finance returns for a given instrument, namely, the Open, High, Low, Close and Adj Close prices for each date. The second dimension contain the dates. The third one contains the instrument identifier.
    
    Google has made fantastic strides in providing us clean data. NaN values from holidays were already accounted for.

In [29]:
panel_data.head(10) #Read first 10 lines

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,225.830002,223.880005,225.039993,225.240005,91366500.0,204.625153
2017-01-04,226.750000,225.610001,225.619995,226.580002,78744400.0,205.842529
2017-01-05,226.580002,225.479996,226.270004,226.399994,78379000.0,205.679031
2017-01-06,227.750000,225.899994,226.529999,227.210007,71559900.0,206.414886
2017-01-09,227.070007,226.419998,226.910004,226.460007,46939700.0,205.733505
2017-01-10,227.449997,226.009995,226.479996,226.460007,63771900.0,205.733505
2017-01-11,227.100006,225.589996,226.360001,227.100006,74650000.0,206.314926
2017-01-12,226.750000,224.960007,226.500000,226.529999,72113200.0,205.797104
2017-01-13,227.399994,226.690002,226.729996,227.050003,62717900.0,206.269501


Now we want to consider are there any redundant or additional features that we may want to implement into the model. When it comes to trading strategies, there are an abundance of indicators that investors may choose from. 

One of the most commonly used indicators is the Moving Average (MA) over a set period of time. For example - the 20 day moving average would be referred to as the 20DMA.

    Pandas has a built-in rolling() function for Series which returns a rolling object for a user-defined window, e.g. 20 days. We will use the closing prices to calculate the 20DMA, 60DMA and 100DMA:

In [30]:
spy_close = panel_data['Close']

spy_20dma = spy_close.rolling(window=20).mean() # 20 Day Moving Average
spy_60dma = spy_close.rolling(window=60).mean() # 60 day
spy_100dma = spy_close.rolling(window=100).mean() # 100 day

print(spy_20dma)

Date
2017-01-03           NaN
2017-01-04           NaN
2017-01-05           NaN
2017-01-06           NaN
2017-01-09           NaN
                 ...    
2022-06-08    403.206001
2022-06-09    403.640501
2022-06-10    403.513501
2022-06-13    402.177501
2022-06-14    400.866501
Name: Close, Length: 1372, dtype: float64


Looking at the Series returned from spy_20dma, the first 20 values are expectededly NaN because 20 values are needed to calcultate the 20DMA. Subsequently, I removed the first 100 samples from the data set to make sure every input has values for 20DMA, 60DMA, and 100DMA.

Additionally, I feel that the date is valuable for model training, however the object would need to be an integer or float to allow for Python's correct interpretation within machine learning models. Thus, I converted the date values to Unix Epoch by writing the following script:


In [31]:
from datetime import datetime

spyData = pd.read_csv('spyData.csv') # Reading from data that was saved to a csv from panel_data earlier

def dateArrange(data):
    dates = data['Date'].to_list()
    epochDates = []
    for i in dates:
        splitDate = i.split('/')
        newDate = splitDate[2] + ',' + splitDate[0] + ',' + splitDate[1] + ',0,0'
        epochDate = datetime(int(splitDate[2]),int(splitDate[0]),int(splitDate[1]),0,0).timestamp()
        epochDates.append(epochDate)
    data['epochDates'] = epochDates

dateArrange(spyData)

spyData['epochDates'].head()

0    1.495685e+09
1    1.495771e+09
2    1.496117e+09
3    1.496203e+09
4    1.496290e+09
Name: epochDates, dtype: float64

In [32]:
from sklearn.model_selection import train_test_split
